In [135]:
import tensorflow
from tensorflow import keras
from keras.callbacks import EarlyStopping
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import joblib

In [5]:
data = pd.read_csv('./lvl_stress_studenti.csv')

In [19]:
x = data.drop('How would you rate your stress levels?', axis=1)
y = data['How would you rate your stress levels?']
for i in range(len(y)):
    y[i] = y[i] - 1
print(y.unique())

[2 1 3 0 4]


In [51]:
x_pack, x_test, y_pack, y_test = train_test_split(x, y, test_size=0.2, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(x_pack, y_pack, test_size=0.2, stratify=y_pack)

In [115]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [117]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
])

F:\virtual_machine\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [119]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [121]:
es = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    restore_best_weights=True
)

In [123]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, callbacks=[es])

Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.1717 - loss: 1.6111 - val_accuracy: 0.3452 - val_loss: 1.5524
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4563 - loss: 1.4869 - val_accuracy: 0.3810 - val_loss: 1.4939
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4750 - loss: 1.4119 - val_accuracy: 0.4048 - val_loss: 1.4479
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4998 - loss: 1.3510 - val_accuracy: 0.4167 - val_loss: 1.4174
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5109 - loss: 1.2898 - val_accuracy: 0.4405 - val_loss: 1.3899
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5495 - loss: 1.2506 - val_accuracy: 0.4405 - val_loss: 1.3656
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5748 - loss: 1.2013 - val_accuracy: 0.5000 - val_loss: 1.3341
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5678 - loss: 1.2172 - val_accuracy: 0.5119 - 

In [127]:
train_loss, train_accuracy = model.evaluate(x_train, y_train)
val_loss, val_accuracy = model.evaluate(x_val, y_val)
test_loss, test_accuracy = model.evaluate(x_test, y_test)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9548 - loss: 0.1391 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9330 - loss: 0.1889
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9610 - loss: 0.2003 


In [133]:
MODEL_DIR = './'
MODEL_FILENAME = 'stress-detector.h5'
FULL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)
model.save(FULL_PATH, save_format='h5')
print('modello compilato')

modello compilato


In [137]:
joblib.dump(scaler, 'scaler.pkl')
print('scaler esportato')

scaler esportato
